# Creating a basic content-based recommender system based on Food2Vec

In [2]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from matplotlib import pyplot

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\quynh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
!pip install pickle
import pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [4]:
data = pickle.load(open("recipe_ingredient_bows.pkl", "rb"))

In [5]:
# Remove sotpwords
def remove_stopwords(text):
    text = text.split()
    text = map(lambda t: t.lower(), text)
    stopword_set = set(stopwords.words("english"))
    text = [word for word in text if not word in stopword_set]
    return " ".join(text)

In [6]:
# Remove punctuation
def remove_punctuation(text):
    token = RegexpTokenizer(r'\w+')
    text = token.tokenize(text)
    return " ".join(text)

In [8]:
df_raw_recipes = pd.read_csv('./archive/RAW_recipes.csv')

In [9]:
df_raw_recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [14]:
df_raw_recipes.drop(columns=['contributor_id', 'submitted'], inplace=True)

In [20]:
df_raw_recipes.head()

,name,id,minutes,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [21]:
model = Word2Vec(data, size=5, window=1000 * len(data), min_count=1, workers=4)
word_vectors = model.wv
X = model[model.wv.vocab]

C:\Users\quynh\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


# Step 2: Create a vector representation based off the words

In [63]:
count = CountVectorizer()
df_copy = df_raw_recipes[0:20001]
count_matrix = count.fit_transform(df_copy['ingredients'])
count_matrix

<20001x2446 sparse matrix of type '<class 'numpy.int64'>'
	with 301667 stored elements in Compressed Sparse Row format>

In [64]:
# Take the cosine similarity matrix
print(df_raw_recipes['ingredients'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)

0         ['winter squash', 'mexican seasoning', 'mixed ...
1         ['prepared pizza crust', 'sausage patty', 'egg...
2         ['ground beef', 'yellow onions', 'diced tomato...
3         ['spreadable cheese with garlic and herbs', 'n...
4         ['tomato juice', 'apple cider vinegar', 'sugar...
                                ...                        
231632    ['celery', 'onion', 'green sweet pepper', 'gar...
231633    ['paprika', 'salt', 'garlic powder', 'onion po...
231634    ['hard-cooked eggs', 'mayonnaise', 'dijon must...
231635    ['butter', 'eagle brand condensed milk', 'ligh...
231636    ['granulated sugar', 'shortening', 'eggs', 'fl...
Name: ingredients, Length: 231637, dtype: object
[[1.         0.09090909 0.05598925 ... 0.12573892 0.14625448 0.06917145]
 [0.09090909 1.         0.1119785  ... 0.25147785 0.14625448 0.27668579]
 [0.05598925 0.1119785  1.         ... 0.30976124 0.40533961 0.29821003]
 ...
 [0.12573892 0.25147785 0.30976124 ... 1.         0.15171652 0.2391

In [65]:
recipe_series = pd.Series(df_copy['name'])

# Step 3: Test Recommender (Basic)

In [66]:
def recommend_recipes(name, n = 10, cosine_sim = cosine_sim):
    recommend_recipes = list()
    
    # Get the index of the recipe name
    idx = recipe_series[recipe_series == name].index[0]
    
    # Get the similarity score
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending=False)
    
    top_n_indices = list(score_series.iloc[1:n+1].index)
    
    for i in top_n_indices:
        recommend_recipes.append(list(df_copy['name'])[i])
    return recommend_recipes

In [67]:
recommend_recipes('alouette  potatoes')

['barley confetti',
 '3 pepper pasta',
 'asparagus salad with sweet pepper confetti',
 'bean and bell pepper salad',
 'absolutely fabulous roasted vegetetables',
 'balsamic baby carrots',
 'acadia s ratatouille',
 'barefoot contessa s greek panzanella',
 'baked fish with ratatouille',
 'anasazi bean veggie burgers']